In [ ]:
%reload_ext dotenv
%dotenv

### Put single garment on human

In [ ]:
import replicate

input = {
    "garm_img": open("./garments/tops/tshirt.webp", "rb"),
    "human_img": open("./humans/model.jpg", "rb"),
    "garment_des": "purple t-shirt",
    "category": "upper_body",
    # "mask_only": True,
}

result_url = replicate.run(
    "cuuupid/idm-vton:c871bb9b046607b680449ecbae55fd8c6d945e0a1948644bf2361b3d021d3ff4",
    input=input
)
print(result_url)

In [ ]:
import requests

with open("./results/tshirt.jpg", "wb") as f:
    f.write(requests.get(result_url).content)

### Combine garments

In [ ]:
from PIL import Image

human = Image.open('./humans/model.jpg')
print(human.format, human.size, human.mode)

mask_bottom = Image.open('./masks/bottom.jpg')
print(mask_bottom.format, mask_bottom.size, mask_bottom.mode)

mask_top = Image.open('./masks/top.jpg')
print(mask_top.format, mask_top.size, mask_top.mode)

result_joggers = Image.open('./results/joggers.jpg')
print(result_joggers.format, result_joggers.size, result_joggers.mode)

result_jeans = Image.open('./results/jeans.jpg')
print(result_jeans.format, result_jeans.size, result_jeans.mode)

result_sweater = Image.open('./results/sweater.jpg')
print(result_sweater.format, result_sweater.size, result_sweater.mode)

result_tshirt = Image.open('./results/tshirt.jpg')
print(result_tshirt.format, result_tshirt.size, result_tshirt.mode)

In [ ]:
def refine_mask(model, model_garment, vton_mask, threshold=1000):
    # compute the difference between the two images
    mask = Image.new("L", model.size)
    for x in range(model.size[0]):
        for y in range(model.size[1]):
            r1, g1, b1 = model.getpixel((x, y))
            r2, g2, b2 = model_garment.getpixel((x, y))
            if ((r2 - r1) ** 2 + (g2 - g1) ** 2 + (b2 - b1) ** 2) > threshold:
                mask.putpixel((x, y), 255)

    # intersect mask with vton mask
    for x in range(model.size[0]):
        for y in range(model.size[1]):
            if vton_mask.getpixel((x, y)) == 0:
                mask.putpixel((x, y), 0)

    return mask

In [ ]:
def combine_garments(human, result_bottom, result_top, mask_top, mask_bottom):
    result = Image.new("RGBA", human.size)
    refined_mask = refine_mask(human, result_top, mask_top)
    result.paste(result_bottom, (0, 0))
    result.paste(result_top, (0, 0), refined_mask)
    return result

In [ ]:
combine_garments(human, result_joggers, result_sweater, mask_top, mask_bottom)

In [ ]:
combine_garments(human, result_jeans, result_tshirt, mask_top, mask_bottom)

In [ ]:
combine_garments(human, result_jeans, result_sweater, mask_top, mask_bottom)

In [ ]:
combine_garments(human, result_joggers, result_tshirt, mask_top, mask_bottom)